In [ ]:
# Import required packages
import gc
import os
import shutil

import numpy as np
import progressbar as pb
import sklearn.utils as sku

import Config as conf
import CSV as csv

In [ ]:
# Import & shuffle CSV data
xx, yy = csv.getCSV()
xx, yy = sku.shuffle(xx, yy, random_state=0)
xx = xx[..., np.newaxis]
xo, yo = xx, yy

In [ ]:
# Import Keras
from keras.applications.vgg19 import VGG19
import keras.callbacks as kc
import keras.layers as kl
import keras.models as km
import keras.optimizers as ko
import keras.utils as ku

In [ ]:
# Setup Keras VGG19 Model
print("GC Collected:", gc.collect())
adam = ko.Adam(lr=conf.LEARNING_RATE, amsgrad=True)
vModel = VGG19(
    input_shape=(conf.WINDOW_SIZE, conf.N_COLUMNS, 1),
    classes=conf.USE_NOACTIVITY and conf.N_CLASSES or conf.N_VALID_CLASSES,
    weights=None,
    include_top=False
)
fltv = kl.Flatten()(vModel.output)
dense = kl.Dense(conf.USE_NOACTIVITY and conf.N_CLASSES or conf.N_VALID_CLASSES, activation='softmax')(fltv)
model = km.Model(inputs=vModel.input, outputs=dense)
model.compile(
    loss="categorical_crossentropy",
    optimizer=adam,
    metrics=["accuracy"]
)
model.summary()

In [ ]:
# Modify activity:non-activity ratio
if conf.USE_NOACTIVITY:
    xx = np.empty([0, conf.WINDOW_SIZE, conf.N_COLUMNS, 1], float)
    yy = np.empty([0, conf.N_CLASSES], float)
    with pb.ProgressBar(max_value=len(yo)) as bar:
        bar.update(0)
        for i in range(len(yo)):
            if yo[i, 0] == 0:
                yy = np.concatenate((yy, yo[i, :][np.newaxis, ...]), axis=0)
                xx = np.concatenate((xx, xo[i, :][np.newaxis, ...]), axis=0)
            bar.update(i)
    print("Found Activity Y:", len(yy))
    noacmx = int(len(yy) * conf.ANOA_RATIO)
    print("Will find", noacmx, "Non-Activity windows")
    noac = 0
    with pb.ProgressBar(max_value=noacmx) as bar:
        bar.update(0)
        for i in range(len(yo)):
            if noac > noacmx:
                break
            if not yo[i, 0] == 0:
                yy = np.concatenate((yy, yo[i, :][np.newaxis, ...]), axis=0)
                xx = np.concatenate((xx, xo[i, :][np.newaxis, ...]), axis=0)
                bar.update(noac)
                noac += 1
    print("Input Ratio Modified, Final Shape: [xx]", xx.shape, "[yy]", yy.shape)

In [ ]:
# Check output directory and prepare tensorboard
outputDir = conf.OUTPUT_PATH.format(conf.LEARNING_RATE, conf.BATCH_SIZE,
                                    "VGG19", "")
if os.path.exists(outputDir):
    shutil.rmtree(outputDir)
os.makedirs(outputDir)
logDir = conf.LOG_PATH.format(conf.LEARNING_RATE, conf.BATCH_SIZE,
                              "VGG19", "")
if os.path.exists(logDir):
    shutil.rmtree(logDir)
os.makedirs(logDir)
tensorboard = kc.TensorBoard(
    log_dir=logDir,
    histogram_freq=0,
    batch_size=conf.BATCH_SIZE,
    write_graph=True,
    write_grads=True,
    write_images=True,
    update_freq=10)
print(
    "If you have tensorboard in this environment, you can type below to see the result in tensorboard:"
)
print("    tensorboard --logdir=" + logDir)
print("Keras checkpoints and final result will be saved in here:")
print("    " + outputDir)

In [ ]:
# Run KFold
xx, yy = sku.shuffle(xx, yy, random_state=0)
for i in range(conf.KFOLD):
    # Run Garbage Collector
    print("Grabage Collecting Time! =>", gc.collect())
    
    # Roll the data
    xx = np.roll(xx, int(len(xx) / conf.KFOLD), axis=0)
    yy = np.roll(yy, int(len(yy) / conf.KFOLD), axis=0)

    # Data separation
    xTrain = xx[int(len(xx) / conf.KFOLD):]
    yTrain = yy[int(len(yy) / conf.KFOLD):]
    xEval = xx[:int(len(xx) / conf.KFOLD)]
    yEval = yy[:int(len(yy) / conf.KFOLD)]

    if not conf.USE_NOACTIVITY:
        # Remove NoActivity from ys
        yTrain = yTrain[:, 1:]
        yEval = yEval[:, 1:]

        # If there exists only one action, convert Y to binary form
        if yEval.shape[1] == 1:
            yTrain = ku.to_categorical(yTrain)
            yEval = ku.to_categorical(yEval)

    # Setup Keras Checkpoint
    checkpoint = kc.ModelCheckpoint(
        outputDir + "Checkpoint_K" + str(i + 1) + "_EPOCH{epoch}_ACC{val_acc:.6f}.h5", period=conf.CP_PERIOD)

    # Fit model (learn)
    print(
        str(i + 1) + " th fitting started. Endpoint is " + str(conf.KFOLD) +
        " th.")

    model.fit(
        xTrain,
        yTrain,
        batch_size=conf.BATCH_SIZE,
        epochs=conf.N_EPOCH,
        verbose=1,
        callbacks=[tensorboard, checkpoint],
        validation_data=(xEval, yEval))  # , validation_freq=2)
print("Epoch completed!")

In [ ]:
# Saving model
print("Saving model & model information...")
modelYML = model.to_yaml()
with open(outputDir + "model.yml", "w") as yml:
    yml.write(modelYML)
modelJSON = model.to_json()
with open(outputDir + "model.json", "w") as json:
    json.write(modelJSON)
model.save(outputDir + "model.h5")
print('Model saved! Congratulations! You finished all processes of ML!')

In [ ]:
# Finished